In [4]:
import numpy as np
import json
import pandas as pd
from datetime import datetime, timedelta, time
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from langdetect import detect
from rapidfuzz import fuzz
import ast
import itertools, pulp

/Applications/anaconda3/envs/DSA_project/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#Build DF used for our code
# Step 1: Load JSON file
with open('/Users/apple/Desktop/DSA_project/DSA2025_birds/flask_app/data/courses.json', encoding="utf-8") as f:
    data = json.load(f)

veranstaltung_list = data['veranstaltungen']['veranstaltung']  # <- FIX IS HERE


df_german = pd.json_normalize(veranstaltung_list, sep='_')

#Step 2: Load translated xlsx file

# read in the data
import openpyxl

df_eng= pd.read_excel(
    "sub_20_eng.xlsx",
    sheet_name="sub_20_eng",
    header=0
    )

In [54]:
#Creating merged_df for better translation
# Step 1: Define the translation mapping (German → English column names)
column_translation = {
    'guid': 'guid',
    'nummer': 'course_number',
    'name': 'course_name',
    'untertitel': 'course_subtitle',
    'bezirk': 'district',
    'veranstaltungsart': 'event_type',
    'minimale_teilnehmerzahl': 'minimum_participants',
    'aktuelle_teilnehmerzahl': 'current_participants',
    'maximale_teilnehmerzahl': 'maximum_participants',
    'anzahl_termine': 'number_of_sessions',
    'beginn_datum': 'start_date',
    'ende_datum': 'end_date',
    'zielgruppe': 'target_group',
    'schlagwort': 'keywords',
    'text': 'description',
    'dvv_kategorie_@version': 'category_version',
    'dvv_kategorie_#text': 'category_label',
    'anmeldung_telefon': 'registration_phone',
    'anmeldung_mail': 'registration_email',
    'anmeldung_link': 'registration_link',
    'ansprechperson_anrede': 'contact_person_salutation',
    'ansprechperson_titel': 'contact_person_title',
    'ansprechperson_name': 'contact_person_last_name',
    'ansprechperson_vorname': 'contact_person_first_name',
    'ansprechperson_telefon': 'contact_person_phone',
    'ansprechperson_mail': 'contact_person_email',
    'ortetermine_adresse': 'locations_address',
    'ortetermine_termin': 'locations_appointments',
    'preis_betrag': 'price_amount',
    'preis_rabatt_moeglich': 'price_discount_possible',
    'preis_zusatz': 'price_additional',
    'dozent_anrede': 'lecturer_salutation',
    'dozent_titel': 'lecturer_title',
    'dozent_name': 'lecturer_last_name',
    'dozent_vorname': 'lecturer_first_name',
    'webadresse_typ': 'website_type',
    'webadresse_name': 'website_last_name',
    'webadresse_uri': 'website_uri',
    'dozent': 'lecturer',
    'merkmale_merkmal_name': 'merkmale_merkmal_last_name',
    'merkmale_merkmal_wert': 'merkmale_merkmal_wert',
    'ortetermine_adresse_lehrstaette': 'locations_address_facility',
    'ortetermine_adresse_plz': 'locations_address_postal_code',
    'ortetermine_adresse_ort': 'locations_address_city',
    'ortetermine_adresse_strasse': 'locations_address_street',
    'ortetermine_adresse_raum': 'locations_address_room',
    'ortetermine_adresse_laengengrad': 'locations_address_longitude',
    'ortetermine_adresse_breitengrad': 'locations_address_latitude',
    'ortetermine_adresse_behindertenzugang': 'locations_address_accessible',
    'ortetermine_termin_wochentag': 'locations_appointments_weekday',
    'ortetermine_termin_beginn_datum': 'locations_appointments_start_date',
    'ortetermine_termin_beginn_uhrzeit': 'locations_appointments_start_time',
    'ortetermine_termin_ende_uhrzeit': 'locations_appointments_end_time'
}

# Step 2: Filter and rename columns in df_german
available_cols = [col for col in column_translation if col in df_german.columns]
df_german_renamed = df_german[available_cols].rename(columns={k: column_translation[k] for k in available_cols})

# Step 3: Ensure guid is string type in both DataFrames
df_eng['guid'] = df_eng['guid'].astype(str)
df_german_renamed['guid'] = df_german_renamed['guid'].astype(str)

# Step 4: Merge on 'guid'
df_merged = df_eng.merge(df_german_renamed, on='guid', how='left', suffixes=('', '_de'))

# Step 5: Overwrite all columns from df_german EXCEPT 'course_name' and 'category_label'
protected = ['course_name', 'category_label']

for col in df_german_renamed.columns:
    if col == 'guid' or col in protected:
        continue
    if f"{col}_de" in df_merged.columns:
        df_merged[col] = df_merged[f"{col}_de"]

# Step 6: Drop all helper *_de columns
df_merged.drop(columns=[col for col in df_merged.columns if col.endswith('_de')], inplace=True)

#also bring the course name in german
df_merged = df_merged.merge(df_german[['guid', 'name']].rename(columns={'name': 'course_name_german'}), on='guid', how='left')



Don't run below traslator file, use saved traslation mapping, instead, intructions for use given

In [ ]:
from deep_translator import GoogleTranslator
from tqdm import tqdm  # progress bar

# Define translation function
def translate_to_english(text):
    try:
        if pd.isna(text):
            return None
        return GoogleTranslator(source='de', target='en').translate(text)
    except Exception:
        return None

# Optional: deduplicate and cache to avoid repeated translations
unique_names = df_merged['course_name_german'].dropna().unique()
translation_map = {name: translate_to_english(name) for name in tqdm(unique_names)}




100%|██████████| 1352/1352 [12:13<00:00,  1.84it/s]


Run the below chunk!

In [51]:
#saving translation map
with open("translation_map.json", "w", encoding="utf-8") as f:
    json.dump(translation_map, f, ensure_ascii=False, indent=2)

# Load later
with open("translation_map.json", "r", encoding="utf-8") as f:
    translation_map = json.load(f)

In [55]:
# Apply to DataFrame
df_merged['course_name_translated'] = df_merged['course_name_german'].map(translation_map)
df_merged['maximum_participants'] = pd.to_numeric(df_merged['maximum_participants'], errors='coerce')
df_merged['current_participants'] = pd.to_numeric(df_merged['current_participants'], errors='coerce')
df_merged['minimum_participants'] = pd.to_numeric(df_merged['minimum_participants'], errors='coerce')

# Apply transformations
df_merged['prop_occupancy_left'] = (df_merged['maximum_participants'] - df_merged['current_participants']) / df_merged['maximum_participants']
df_merged['prop_minimum_to_reach'] = (df_merged['minimum_participants'] - df_merged['current_participants']) / df_merged['minimum_participants']
df_merged['prop_minimum_to_reach'] = df_merged['prop_minimum_to_reach'].clip(lower=0)

np.random.seed(42)
df_merged['number_of_women'] = df_merged['current_participants'].apply(lambda x: np.random.randint(0, x + 1) if x > 0 else 0)
df_merged['percent_women'] = np.where(df_merged['current_participants'] > 0,
                                      df_merged['number_of_women'] / df_merged['current_participants'], 0)
df_merged['prop_men'] = np.where(df_merged['current_participants'] > 0,
                                 (df_merged['current_participants'] - df_merged['number_of_women']) / df_merged['current_participants'], 0)

df_merged['sponsored'] = np.random.choice([1, 0], size=len(df_merged), p=[0.25, 0.75])
df_merged['gap_to_80_percent_women'] = 0.8 - df_merged['percent_women']
df_merged['gap_to_80_percent_men'] = 0.8 - df_merged['prop_men']

# One-hot encode target groups
if 'target_group' in df_merged.columns:
    df_merged['target_group_raw'] = df_merged['target_group']  # make a copy before one-hot encoding

    target_groups = df_merged['target_group_raw'].dropna().unique()
    for group in target_groups:
        col = f"target_group_{group}"
        df_merged[col] = df_merged['target_group_raw'].apply(lambda x: 1 if x == group else 0)


# Ensure price is numeric
df_merged['price_amount'] = pd.to_numeric(df_merged.get('price_amount', np.nan), errors='coerce')

#Remove courses that have no occupancy left
df_merged = df_merged[df_merged['prop_occupancy_left'] > 0]




In [56]:
df_merged.columns.tolist()

['guid',
 'course_number',
 'course_name',
 'course_subtitle',
 'district',
 'event_type',
 'minimum_participants',
 'current_participants',
 'maximum_participants',
 'number_of_sessions',
 'start_date',
 'end_date',
 'target_group',
 'keywords',
 'description',
 'category_version',
 'category_text',
 'registration_phone',
 'registration_email',
 'registration_link',
 'contact_person_salutation',
 'contact_person_title',
 'contact_person_last_name',
 'contact_person_first_name',
 'contact_person_phone',
 'contact_person_email',
 'locations_address',
 'locations_appointments',
 'price_amount',
 'price_discount_possible',
 'price_additional',
 'lecturer_salutation',
 'lecturer_title',
 'lecturer_last_name',
 'lecturer_first_name',
 'website_type',
 'website_last_name',
 'website_uri',
 'lecturer',
 'merkmale_merkmal_last_name',
 'merkmale_merkmal_wert',
 'locations_address_facility',
 'locations_address_postal_code',
 'locations_address_city',
 'locations_address_street',
 'locations_add

In [57]:
def safe_parse(x):
    try:
        return ast.literal_eval(x) if isinstance(x, str) else x
    except:
        return x

def flatten_keywords(x):
    x = safe_parse(x)
    if isinstance(x, list):
        return ', '.join(map(str, x))
    return str(x)
df_merged['keywords_clean'] = df_merged['keywords'].apply(flatten_keywords)
# Save the cleaned DataFrame
df_merged.to_csv("df_merged_preprocessed.csv", index=False)

In [58]:
from sentence_transformers import SentenceTransformer


# 1. Load German-compatible model
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
model.save("saved_sentence_transformer_model")

# 2. Prepare search text for all courses (if not already done)
df_merged['search_text'] = (
    df_merged['course_name_german'].fillna('') + ' ' + df_merged['course_subtitle'].fillna('')+ ' ' +
    df_merged['keywords_clean'].fillna('') 
)

# 3. Embed the full catalog (one-time step)----should be done for the whole dataset in the dataprep stage
course_embeddings = model.encode(df_merged['search_text'].tolist(), show_progress_bar=True)
np.save("course_embeddings.npy", course_embeddings)

Batches: 100%|██████████| 48/48 [00:03<00:00, 13.00it/s]


In [61]:
def translate_query_to_german(query):
    try:
        return GoogleTranslator(source='auto', target='de').translate(query)
    except Exception as e:
        raise ValueError(f"Translation failed: {e}")

In [62]:
def get_course_matches(user_query, df, model, course_embeddings, user_budget, top_n=20, similarity_threshold=0.45):
    import numpy as np

    if not isinstance(user_query, str) or not user_query.strip():
        raise ValueError("Invalid input. Please provide a non-empty search query.")

    # Detect language and translate if needed
    try:
        detected_lang = detect(user_query)
    except Exception:
        detected_lang = "en"

    translated_query = user_query if detected_lang == 'de' else translate_query_to_german(user_query)
    search_tokens = translated_query.lower().split()

    # Fuzzy keyword matching
    def fuzzy_token_match(text):
        if pd.isna(text): return False
        text = text.lower()
        return any(fuzz.partial_ratio(token, text) >= 50 for token in search_tokens)

    # Step 1: Basic fuzzy filter
    df_filtered = df[
        df['course_name_german'].apply(fuzzy_token_match) |
        df['course_name_translated'].apply(fuzzy_token_match)
    ]

    if df_filtered.empty:
        raise ValueError("No courses matched any extracted keyword tokens. Try a different query.")

    # Step 2: Semantic similarity
    query_embedding = model.encode([translated_query])
    filtered_embeddings = model.encode(df_filtered['search_text'].tolist())
    similarities = cosine_similarity(query_embedding, filtered_embeddings)[0]

    df_filtered = df_filtered.copy()
    df_filtered['semantic_score'] = similarities
    df_filtered = df_filtered[similarities >= similarity_threshold]

    if df_filtered.empty:
        raise ValueError("Courses matched the keywords, but were semantically too distant.")

    # Step 3: Filter by budget range (±30%)
    min_price = user_budget * 0.7
    max_price = user_budget * 1.3
    df_filtered = df_filtered[df_filtered['price_amount'].between(min_price, max_price)]

    if df_filtered.empty:
        raise ValueError("No matches for this price filter, please remove filter to see all matches.")

    # Step 4: Compute price penalty
    def compute_penalty(price):
        if price >= user_budget:
            penalty = (1.3 * user_budget - price) / (0.3 * user_budget)
        else:
            penalty = (price - 0.7 * user_budget) / (0.3 * user_budget)
        return max(0, min(1, penalty))  # ensure it's between 0 and 1

    df_filtered['price_penalty_scaled'] = df_filtered['price_amount'].apply(compute_penalty)

    # Step 5: Compute weighted score
    avg_sem_score = df_filtered['semantic_score'].nlargest(top_n).mean()
    df_filtered['weighted_price_penalty'] = df_filtered['price_penalty_scaled'] * avg_sem_score

    # Step 6: Final score = 65% semantic + 35% (1 - penalty weight)
    df_filtered['final_score'] = (
        0.65 * df_filtered['semantic_score'] +
        0.35 * (1-df_filtered['weighted_price_penalty'])
    )

    # Step 7: Sort and rank
    top_courses = df_filtered.sort_values(by='final_score', ascending=False).head(top_n).copy()
    top_courses['final_rank'] = np.arange(1, len(top_courses) + 1)

    return top_courses


In [ ]:

TARGET_GROUP_MAPPING = {
    "People with a migration background": "Menschen mit Migrationshintergrund",
    "Illiterate people": "Analphabet/inn/en",
    "Women": "Frauen",
    "People with disabilities": "Menschen mit Behinderung",
    "Older adults / older people": "Ältere",
    "Other target groups": "Andere Adressaten–gruppen",
    "Children": "Kinder",
    "Adolescents / young people": "Jugendliche"
}

class PlatformPreferenceRanker:
    def __init__(self, user_gender: str, selected_target_groups: list):
        self.user_gender = user_gender.lower()
        self.gender_col = 'gap_to_80_percent_women' if self.user_gender == 'female' else 'gap_to_80_percent_men'

        if not self.user_gender or selected_target_groups is None:
            raise ValueError("Please fill in both gender and target group fields before proceeding.")

        self.selected_target_groups = selected_target_groups.copy()
        if self.user_gender == 'female' and "Women" not in self.selected_target_groups:
            self.selected_target_groups.append("Women")

    def rank(self, matched_df: pd.DataFrame):
        df = matched_df.copy()

        core_cols = ['prop_occupancy_left', 'prop_minimum_to_reach', self.gender_col]
        df['numeric_score'] = df[core_cols].sum(axis=1)
        df = df.sort_values(by='numeric_score', ascending=False).reset_index(drop=True)
        df['rank_index'] = df.index

        binary_cols = ['sponsored']
        for tg in self.selected_target_groups:
            german = TARGET_GROUP_MAPPING.get(tg)
            if german:
                col = f"target_group_{german}"
                if col in df.columns:
                    binary_cols.append(col)
                else:
                    df[col] = 0
                    binary_cols.append(col)

        df['binary_sum'] = df[binary_cols].sum(axis=1)

        max_score = df['numeric_score'].max()
        min_score = df['numeric_score'].min()
        total = len(df)

        def calculate_weight(row):
            if row['rank_index'] > 0:
                return ((max_score - row['numeric_score']) / row['rank_index']) * 1.05
            return ((max_score - min_score) / total) * 1.05

        df['weight'] = df.apply(calculate_weight, axis=1)
        df['binary_boost'] = df['weight'] * df['binary_sum']
        df['final_score_platform'] = df['numeric_score'] + df['binary_boost']
        df = df.sort_values(by='final_score_platform', ascending=False).reset_index(drop=True)

        return df


In [65]:

class ConsensusRanker:
    def __init__(self, user_df: pd.DataFrame, platform_df: pd.DataFrame):
        self.user_df = user_df
        self.platform_df = platform_df
        self.user_order = list(user_df['guid'])
        self.platform_order = list(platform_df['guid'])

        if set(self.user_order) != set(self.platform_order):
            raise ValueError("user_order and platform_order must contain the same GUIDs")

    def compute_consensus(self):
        courses = self.user_order
        user_rank = {c: i for i, c in enumerate(self.user_order)}
        platform_rank = {c: i for i, c in enumerate(self.platform_order)}

        w_user, w_plat = 0.5, 0.5

        margins = {}
        for i, j in itertools.combinations(courses, 2):
            sign_user = 1 if user_rank[i] < user_rank[j] else -1
            sign_plat = 1 if platform_rank[i] < platform_rank[j] else -1
            vote = w_user * sign_user + w_plat * sign_plat

            if vote > 0:
                margins[(i, j)] = abs(vote)
            elif vote < 0:
                margins[(j, i)] = abs(vote)
            else:
                margins[(i, j)] = 0.1
                margins[(j, i)] = 0.1

        model = pulp.LpProblem("Kemeny", pulp.LpMinimize)
        x = pulp.LpVariable.dicts('x', (courses, courses), 0, 1, cat='Binary')
        model += pulp.lpSum(weight * x[j][i] for (i, j), weight in margins.items())

        for i, j in itertools.permutations(courses, 2):
            model += x[i][j] + x[j][i] == 1

        for i, j, k in itertools.permutations(courses, 3):
            model += x[i][j] + x[j][k] + x[k][i] >= 1

        model.solve(pulp.PULP_CBC_CMD(msg=False))

        consensus_order = sorted(
            courses,
            key=lambda c: sum(x[c][d].value() for d in courses if d != c),
            reverse=True
        )

        return consensus_order

    def get_ranked_df(self):
        consensus_order = self.compute_consensus()
        df = self.user_df.copy()
        df['__rank__'] = df['guid'].apply(lambda x: consensus_order.index(x))
        df.sort_values(by='__rank__', inplace=True)
        df.drop(columns='__rank__', inplace=True)
        return df


In [69]:
user_query="French classes for beginners"
user_budget=100
matcher = CourseMatcher(df=df_merged, model=model, course_embeddings=course_embeddings)
final_matches_df = matcher.run(user_query=user_query, user_budget=user_budget)

In [70]:
user_gender = "female"
user_target_groups = ["People with a migration background"] 

ranker = PlatformPreferenceRanker(user_gender=user_gender, selected_target_groups=user_target_groups)
platform_ranked_df = ranker.rank(final_matches_df)

In [72]:
consensus = ConsensusRanker(final_matches_df, platform_ranked_df)
final_output_df = consensus.get_ranked_df()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [73]:
final_output_df[['course_name_german', 'course_name_translated', 'price_amount',
                 'gap_to_80_percent_women', 'prop_occupancy_left',
                 'prop_minimum_to_reach', 'gap_to_80_percent_men', 'target_group']]


,course_name_german,course_name_translated,price_amount,gap_to_80_percent_women,prop_occupancy_left,prop_minimum_to_reach,gap_to_80_percent_men,target_group
761,Französisch A2.1-A2.4 Auffrischung und Wiedere...,French A2.1 -A2.4 refreshment and re -entry - ...,76.70,0.133333,0.400000,0.142857,0.466667,None
1520,Französisch B1 Auffrischung und Konversation O...,French B1 refreshment and conversation online,74.40,0.466667,0.357143,0.000000,0.133333,None
1476,Französisch A1.3,French A1.3,90.98,0.800000,0.937500,0.857143,-0.200000,None
1453,"Französisch A2.2, Lektionen 4-6 - Onlinekurs","French A2.2, lessons 4-6 - online course",82.20,0.133333,0.400000,0.000000,0.466667,None
654,Porträtzeichnen und -malen,Portrait pages and painting,126.13,0.800000,1.000000,1.000000,0.800000,None
1494,Französisch B1.2 Kompakt in 6 Wochen Online,French B1.2 compact online in 6 weeks,88.68,0.500000,0.285714,0.000000,0.100000,None
1341,"Spanisch A1.1, Lektionen 1-4 - Intensivkurs","Spanish A1.1, lesson 1-4 - intensive course",118.50,0.800000,1.000000,1.000000,0.800000,None
1342,"Spanisch A1.2, Lektionen 5-8 - Intensivkurs","Spanish A1.2, lesson 5-8 - intensive course",118.50,0.600000,0.583333,0.166667,0.000000,None
861,Tango Argentino - Kurs für Paare,Tango Argentino - course for couples,75.50,0.527273,0.388889,0.000000,0.072727,None
851,Tango Argentino - Grundkurs für Paare,Tango Argentino - basic course for couples,75.50,0.133333,0.500000,0.250000,0.466667,None
